In [ ]:
import torch
from hydra import compose, initialize
from PIL import Image 

from yolo import AugmentationComposer, bbox_nms, Config, create_model, custom_logger, draw_bboxes

In [ ]:
CONFIG_PATH = "../yolo/config"
CONFIG_NAME = "config"

DEVICE = 'cuda:0'
# If weight not exist there, it will try to download it

WEIGHT_PATH = '../weights/v9-cnw.pt' 
IMAGE_PATH = '../demo/images/inference/image.png'

custom_logger()
device = torch.device(DEVICE)
image = Image.open(IMAGE_PATH)

In [ ]:
with initialize(config_path=CONFIG_PATH, version_base=None, job_name="notebook_job"):
    cfg: Config = compose(config_name=CONFIG_NAME, overrides=["task=inference", f"task.data.source={IMAGE_PATH}"])
    model = create_model(cfg.model, WEIGHT_PATH).to(device)
    transform = AugmentationComposer([], cfg.image_size)

In [ ]:
image, bbox = transform(image, torch.zeros(0, 5))
image = image.to(device)[None]

In [ ]:
with torch.no_grad():
    predict = model(image)
predict_box = bbox_nms(predict[1][0], cfg.task.nms)
draw_bboxes(image, predict_box, save_path='../demo/images/output/', idx2label=cfg.class_list)

Sample Output:

![image](../demo/images/output/visualize.png)